In [ ]:
%%capture
!pip install fastapi uvicorn python-multipart langchain langchain-community pypdf sqlalchemy pydantic python-jose[cryptography] passlib[bcrypt] llama-index faiss-gpu

In [ ]:
!pip install pyngrok
!ngrok authtoken [ngrok-token]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile main.py
from fastapi import FastAPI, File, UploadFile, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional, List
from datetime import datetime
import os
import shutil
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from google.oauth2 import service_account
import vertexai
from vertexai.generative_models import GenerativeModel

# Initialize FastAPI app
app = FastAPI(title="PDF Q&A API")

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Google Cloud credentials setup
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "YOUR_SERVICE_KEY.JSON"
credentials = service_account.Credentials.from_service_account_file(
    "YOUR_SERVICE_KEY.JSON"
)
vertexai.init(project="YOUR_GCP_PROJECT_ID", location="asia-south1", credentials=credentials)

# Database setup
SQLALCHEMY_DATABASE_URL = "sqlite:///./pdf_qa.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database Models
class Document(Base):
    __tablename__ = "documents"

    id = Column(Integer, primary_key=True, index=True)
    filename = Column(String, unique=True, index=True)
    upload_date = Column(DateTime, default=datetime.utcnow)
    file_path = Column(String)
    vector_store_path = Column(String)

# Create database tables
Base.metadata.create_all(bind=engine)

# Dependency to get database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Pydantic models for request/response
class QuestionRequest(BaseModel):
    document_id: int
    question: str

class QuestionResponse(BaseModel):
    answer: str
    document_id: int
    question: str

# File handling functions
def save_upload_file(upload_file: UploadFile, destination: str) -> str:
    try:
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(upload_file.file, buffer)
    finally:
        upload_file.file.close()
    return destination

def process_pdf(file_path: str, document_id: int):
    # Load PDF
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
    texts = text_splitter.split_documents(documents)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings()
    vector_store = FAISS.from_documents(texts, embeddings)

    # Save vector store
    vector_store_path = f"vector_stores/document_{document_id}"
    os.makedirs("vector_stores", exist_ok=True)
    vector_store.save_local(vector_store_path)

    return vector_store_path

# API endpoints
@app.post("/upload/")
async def upload_file(
    file: UploadFile = File(...),
    db: Session = Depends(get_db)
):
    if not file.filename.endswith('.pdf'):
        raise HTTPException(status_code=400, detail="Only PDF files are allowed")

    # Create uploads directory if it doesn't exist
    os.makedirs("uploads", exist_ok=True)

    # Save file
    file_path = f"uploads/{file.filename}"
    save_upload_file(file, file_path)

    # Create document record
    db_document = Document(
        filename=file.filename,
        file_path=file_path
    )
    db.add(db_document)
    db.commit()
    db.refresh(db_document)

    # Process PDF and save vector store
    try:
        vector_store_path = process_pdf(file_path, db_document.id)
        db_document.vector_store_path = vector_store_path
        db.commit()
    except Exception as e:
        db.delete(db_document)
        db.commit()
        os.remove(file_path)
        raise HTTPException(status_code=500, detail=f"Error processing PDF: {str(e)}")

    return {"document_id": db_document.id, "filename": file.filename}

@app.post("/question/", response_model=QuestionResponse)
async def ask_question(
    question_request: QuestionRequest,
    db: Session = Depends(get_db)
):
    # Get document
    document = db.query(Document).filter(Document.id == question_request.document_id).first()
    if not document:
        raise HTTPException(status_code=404, detail="Document not found")

    try:
        # Load vector store
        embeddings = HuggingFaceEmbeddings()
        vector_store = FAISS.load_local(document.vector_store_path, embeddings,
            allow_dangerous_deserialization=True)

        # Get relevant documents
        docs = vector_store.similarity_search(question_request.question, k=6)

        # Prepare text context from relevant documents
        context = " ".join(doc.page_content for doc in docs)

        # Prepare input for Gemini
        input_text = f"\nContext: {context}\n\nQuestion: {question_request.question}\n\nPlease provide a comprehensive 4-5 line answer based on the given context."

        # Initialize Gemini model
        model = GenerativeModel("gemini-1.5-flash-002")

        # Generate response
        response = model.generate_content(input_text)
        answer = response.text.strip()

        return QuestionResponse(
            answer=answer,
            document_id=question_request.document_id,
            question=question_request.question,
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing question: {str(e)}")

@app.get("/documents/")
async def list_documents(db: Session = Depends(get_db)):
    documents = db.query(Document).all()
    return documents

Overwriting main.py


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HUGGING_FACE_API_TOKEN"

In [ ]:
#Run the server with ngrok
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import os
from threading import Thread

# Enable nested asyncio support
nest_asyncio.apply()

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Run the FastAPI app
os.system('python -m uvicorn main:app --host 0.0.0.0 --port 8000')

Public URL: https://b681-34-148-230-12.ngrok-free.app


2